# Mass balance and mass action equations for chemical equilibrium calculations

In [36]:
from reaktoro import *
db = Database("supcrt98.xml")
editor = ChemicalEditor(db)
editor.addAqueousPhaseWithElements("H O C")
editor.addGaseousPhase(["H2O(g)", "CO2(g)"])

system = ChemicalSystem(editor)

problem = EquilibriumProblem(system)
T = 100 # celsius
P = 50.0 # bars
problem.setTemperature(T, "celsius")
problem.setPressure(P, "bar")
problem.add("H2O", 100, "mol")
problem.add("CO2", 2, "mol")

In [37]:
state = equilibrate(problem)

In [40]:
b = state.elementAmounts()
n = state.speciesAmounts()
print("b = ", b)
print("n = ", n)
A = system.formulaMatrix()
print("A =\n", A)

b =  [2.00000000e+00 2.00000000e+02 1.04000000e+02 3.95640487e-20]
n =  [6.30809566e-22 7.79862569e-01 1.77221267e-10 8.03967468e-04
 8.64975919e-22 9.99673729e+01 2.02278061e-21 8.03964613e-04
 4.86284124e-22 4.41223813e-20 2.50031925e-09 3.18231510e-02
 1.21933347e+00]
A =
 [[ 1.  1.  1.  0.  0.  0.  0.  1.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  1.  2.  2.  2.  1.  1.  0.  1.  2.  0.]
 [ 1.  2.  3.  0.  0.  1.  2.  3.  2.  2.  1.  1.  2.]
 [ 0.  0. -2.  1.  0.  0.  0. -1. -1.  0. -1.  0.  0.]]


Residual evalaution:

In [41]:
r = b - np.dot(A, n)
import numpy as np
r_norm = np.linalg.norm(r)
print("||r|| = ", r_norm)

||r|| =  3.1776437161565096e-14


How much of the CO2(g) is dissolved as CO2(aq)?

In [43]:
print(f"CO2(aq) amount is {state.speciesAmount('CO2(aq)'):6.4e} mol")

CO2(aq) amount is 7.7986e-01 mol


How much of the HO2(l) is evaporate as HO2(g)?

In [45]:
print(f"HO2(g) amount is {state.speciesAmount('H2O(g)'):6.4e} mol")

HO2(g) amount is 3.1823e-02 mol


What is the amount of H+(aq)?

In [46]:
print(f"H+ amount is {state.speciesAmount('H+'):6.4e} mol")

H+ amount is 8.0397e-04 mol


In [48]:
n, m = A.shape
for i in range(n):
    for j in range(m):
        print(f"{A[i][j]:4.0f}", end=" ")
    print("\n")

   1    1    1    0    0    0    0    1    0    0    0    0    1 

   0    0    0    1    2    2    2    1    1    0    1    2    0 

   1    2    3    0    0    1    2    3    2    2    1    1    2 

   0    0   -2    1    0    0    0   -1   -1    0   -1    0    0 



Rank is the maximal number of linearly independent columns of A, and is iqual to the dimension of the vector space spanned by its rows.

In [50]:
rank = np.linalg.matrix_rank(A)
print("Rank of A is", rank)

Rank of A is 4


The matrix from the lectures:

In [51]:
A_ = [[2, 1, 1, 0, 1, 0, 0, 2],
      [1, 0, 1, 3, 3, 2, 2, 1],
      [0, 0, 0, 1, 1, 1, 1, 0],
      [0, 1, -1, -2, -1, 0, 0, 0]]
print("Rank of A is", np.linalg.matrix_rank(A_))

Rank of A is 3


Which phases exist in equilibrium state? 

In [52]:
phases_names = [phase.name() for phase in system.phases()]
stability_indices = state.phaseStabilityIndices()
print("Phases  : Phase amounts : Stability Indices")
for name, si in zip(phases_names, stability_indices):
    print(f"{name:>7} : {state.phaseAmount(name):13.4f} : {si}")

Phases  : Phase amounts : Stability Indices
Aqueous :      100.7488 : -4.821637332766436e-17
Gaseous :        1.2512 : 0.0


**Note**: The stability index of a *stable phase* is zero (or very close to zero), negative for *under-saturated phase*, and positive for the *over-saturated phase*.

In [53]:
molar_masses = [element.molarMass() for element in system.elements()]
element_names = [element.name() for element in system.elements()]
for name, molar_mass in zip(element_names, molar_masses):
    print(f"{name} : {molar_mass:6.4e} (kg/mol)")

C : 1.2011e-02 (kg/mol)
H : 1.0079e-03 (kg/mol)
O : 1.5999e-02 (kg/mol)
Z : 0.0000e+00 (kg/mol)


In [54]:
for name, amount, molar_mass in zip(element_names, b, molar_masses):
    print(f"{name} : {amount:6.2f} mols and {amount*molar_mass*1e3:8.2f} g")

C :   2.00 mols and    24.02 g
H : 200.00 mols and   201.59 g
O : 104.00 mols and  1663.94 g
Z :   0.00 mols and     0.00 g


What are the amounts of each chemical species 

In [59]:
species_names = [specie.name() for specie in system.species()]
print(species_names)
n = state.speciesAmounts()
print(n)

['CO(aq)', 'CO2(aq)', 'CO3--', 'H+', 'H2(aq)', 'H2O(l)', 'H2O2(aq)', 'HCO3-', 'HO2-', 'O2(aq)', 'OH-', 'H2O(g)', 'CO2(g)']
[6.30809566e-22 7.79862569e-01 1.77221267e-10 8.03967468e-04
 8.64975919e-22 9.99673729e+01 2.02278061e-21 8.03964613e-04
 4.86284124e-22 4.41223813e-20 2.50031925e-09 3.18231510e-02
 1.21933347e+00]


In [60]:
i = 0
for name, amount in zip(species_names, n):
    species_molar_mass = np.dot(A[:, i], molar_masses) * 1e3 # in g
    mass = amount * species_molar_mass
    print(f"{name:>10} : {amount:8.4f} mols and {mass:10.4f} g")
    i += 1

    CO(aq) :   0.0000 mols and     0.0000 g
   CO2(aq) :   0.7799 mols and    34.3214 g
     CO3-- :   0.0000 mols and     0.0000 g
        H+ :   0.0008 mols and     0.0008 g
    H2(aq) :   0.0000 mols and     0.0000 g
    H2O(l) :  99.9674 mols and  1800.9402 g
  H2O2(aq) :   0.0000 mols and     0.0000 g
     HCO3- :   0.0008 mols and     0.0491 g
      HO2- :   0.0000 mols and     0.0000 g
    O2(aq) :   0.0000 mols and     0.0000 g
       OH- :   0.0000 mols and     0.0000 g
    H2O(g) :   0.0318 mols and     0.5733 g
    CO2(g) :   1.2193 mols and    53.6623 g


In [62]:
chemical_properties = system.properties(T, P, n)
print("Chemical potentials of the species:")
for potential, species, index in zip(
    chemical_properties.chemicalPotentials().val,
    system.species(),
    list(range(1, system.numSpecies()+1))
):
    print(f"\u03BC_{index} ({species.name():>8}) = {potential:>12.4f} (J/mol)")

Chemical potentials of the species:
μ_1 (  CO(aq)) = -158834.9796 (J/mol)
μ_2 ( CO2(aq)) = -384019.6818 (J/mol)
μ_3 (   CO3--) = -548111.4799 (J/mol)
μ_4 (      H+) =   -6439.8261 (J/mol)
μ_5 (  H2(aq)) =  -21842.3064 (J/mol)
μ_6 (  H2O(l)) = -235514.9383 (J/mol)
μ_7 (H2O2(aq)) = -170564.6383 (J/mol)
μ_8 (   HCO3-) = -590866.7831 (J/mol)
μ_9 (    HO2-) = -107842.6635 (J/mol)
μ_10 (  O2(aq)) =  -18555.1608 (J/mol)
μ_11 (     OH-) = -174370.4789 (J/mol)
μ_12 (  H2O(g)) = -232865.7609 (J/mol)
μ_13 (  CO2(g)) = -395395.8062 (J/mol)


In [63]:
print("Logarithms of activities of the species:")
for activity, species, index in zip(
    chemical_properties.lnActivities().val,
    system.species(),
    list(range(1, system.numSpecies()+1))
):
    print(f"ln (a_{index}) ({species.name():>10}) = {activity:6.4e}")

Logarithms of activities of the species:
ln (a_1) (    CO(aq)) = -4.9403e+01
ln (a_2) (   CO2(aq)) = -8.3695e-01
ln (a_3) (     CO3--) = -2.3143e+01
ln (a_4) (        H+) = -7.7453e+00
ln (a_5) (    H2(aq)) = -4.9088e+01
ln (a_6) (    H2O(l)) = -1.5899e-05
ln (a_7) (  H2O2(aq)) = -4.8238e+01
ln (a_8) (     HCO3-) = -7.7454e+00
ln (a_9) (      HO2-) = -4.9695e+01
ln (a_10) (    O2(aq)) = -4.5156e+01
ln (a_11) (       OH-) = -2.0426e+01
ln (a_12) (    H2O(g)) = -1.1273e+01
ln (a_13) (    CO2(g)) = -7.6267e+00


In [64]:
mu_H2O = - 242.992 * 1e3 # J / mol
mu_Hplus = 0 # J / mol
mu_OHminus = - 155.559 * 1e3 # J / mol

nu_H2O = 1
nu_Hplus = 1
nu_OHminus = 1

R = 8.314 # J / (mol * K)
T = 100 + 273.15 # K

lnK = - 1 / (R * T) * (nu_Hplus * mu_Hplus
                       + nu_OHminus * mu_OHminus 
                       - nu_H2O * mu_H2O)
print("lnK = ", lnK)

lnK =  -28.182655635655994


Recalling that $log_{10} K = \frac{ln K}{ln 10}$, we obtain:

In [65]:
ln10 = 2.30
logK = lnK / ln10
print("logK = ", logK)

logK =  -12.253328537241737
